In [1]:
from delta import configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession

In [2]:
builder = (SparkSession.builder
           .appName("optimize-delta-table")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c41fc6a7-bc07-4032-97cf-4b411b9f4a83;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 397ms :: artifacts dl 17ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

In [3]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [4]:
# For PySpark:
df = spark.read.format("delta").load("/opt/workspace/data/delta_lake/netflix_titles")

In [5]:
df.show(5)

+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null|United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s2|TV Show|       Blood & Water|               |Ama Qamata, Khosi...| South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|Julien Leclercq|Sami Bouajila, Tr...|         null|Septem

In [6]:
%%sparksql
DESCRIBE HISTORY "/opt/workspace/data/delta_lake/netflix_titles"

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-02-04 17:05:38.590000,null,null,UPDATE,"{'predicate': '[""isnull(director#40)""]'}",null,null,null,2,Serializable,False,"{'numRemovedBytes': '1960782', 'numAddedFiles': '1', 'scanTimeMs': '2915', 'numCopiedRows': '6172', 'executionTimeMs': '5820', 'numAddedChangeFiles': '0', 'numAddedBytes': '1963749', 'numUpdatedRows': '2634', 'numRemovedFiles': '1', 'rewriteTimeMs': '2899'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
2,2024-02-04 17:04:16.904000,null,null,SET TBLPROPERTIES,"{'properties': '{""delta.logRetentionDuration"":""interval 60 days"",""delta.deletedFileRetentionDuration"":""interval 14 days""}'}",null,null,null,1,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
1,2024-02-04 17:03:13.178000,null,null,CREATE OR REPLACE TABLE AS SELECT,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,0,Serializable,False,"{'numOutputRows': '8806', 'numOutputBytes': '1960782', 'numFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2024-02-04 17:03:06.818000,null,null,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,null,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


### Optimize performance with file management - Compaction (bin-packing)

In [7]:
deltaTable = DeltaTable.forPath(spark, "/opt/workspace/data/delta_lake/netflix_titles")  
# For Hive metastore-based tables: deltaTable = DeltaTable.forName(spark, tableName)
deltaTable.optimize().executeCompaction()

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [8]:
%%sparksql
-- Optimizes the path-based Delta Lake table
OPTIMIZE "/opt/workspace/data/delta_lake/netflix_titles" 

path,metrics
file:/opt/workspace/data/delta_lake/netflix_titles,"Row(numFilesAdded=0, numFilesRemoved=0, filesAdded=Row(min=None, max=None, avg=0.0, totalFiles=0, totalSize=0), filesRemoved=Row(min=None, max=None, avg=0.0, totalFiles=0, totalSize=0), partitionsOptimized=0, zOrderStats=None, numBatches=0, totalConsideredFiles=1, totalFilesSkipped=1, preserveInsertionOrder=False, numFilesSkippedToReduceWriteAmplification=0, numBytesSkippedToReduceWriteAmplification=0, startTimeMs=1707066836428, endTimeMs=0, totalClusterParallelism=2, totalScheduledTasks=0, autoCompactParallelismStats=None, deletionVectorStats=None, numTableColumns=12, numTableColumnsWithStats=12)"


### Data skipping - Z-Ordering (multi-dimensional clustering)

In [9]:
deltaTable = DeltaTable.forPath(spark, "/opt/workspace/data/delta_lake/netflix_titles")  # path-based table
# For Hive metastore-based tables: deltaTable = DeltaTable.forName(spark, tableName)
deltaTable.optimize().executeZOrderBy("country")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [10]:
%%sparksql
OPTIMIZE "/opt/workspace/data/delta_lake/netflix_titles" ZORDER BY (country)

path,metrics
file:/opt/workspace/data/delta_lake/netflix_titles,"Row(numFilesAdded=1, numFilesRemoved=1, filesAdded=Row(min=1963749, max=1963749, avg=1963749.0, totalFiles=1, totalSize=1963749), filesRemoved=Row(min=1963749, max=1963749, avg=1963749.0, totalFiles=1, totalSize=1963749), partitionsOptimized=1, zOrderStats=Row(strategyName='all', inputCubeFiles=Row(num=0, size=0), inputOtherFiles=Row(num=1, size=1963749), inputNumCubes=0, mergedFiles=Row(num=1, size=1963749), numOutputCubes=1, mergedNumCubes=None), numBatches=1, totalConsideredFiles=1, totalFilesSkipped=0, preserveInsertionOrder=False, numFilesSkippedToReduceWriteAmplification=0, numBytesSkippedToReduceWriteAmplification=0, startTimeMs=1707066851774, endTimeMs=0, totalClusterParallelism=2, totalScheduledTasks=0, autoCompactParallelismStats=None, deletionVectorStats=None, numTableColumns=12, numTableColumnsWithStats=12)"


In [11]:
%%sparksql
DESCRIBE HISTORY "/opt/workspace/data/delta_lake/netflix_titles"

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2024-02-04 17:14:08.504000,null,null,OPTIMIZE,"{'predicate': '[]', 'zOrderBy': '[""country""]'}",null,null,null,4,SnapshotIsolation,False,"{'numRemovedBytes': '1963749', 'p50FileSize': '1963749', 'p25FileSize': '1963749', 'numAddedFiles': '1', 'minFileSize': '1963749', 'p75FileSize': '1963749', 'numDeletionVectorsRemoved': '0', 'numAddedBytes': '1963749', 'maxFileSize': '1963749', 'numRemovedFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
4,2024-02-04 17:14:01.672000,null,null,OPTIMIZE,"{'predicate': '[]', 'zOrderBy': '[""country""]'}",null,null,null,3,SnapshotIsolation,False,"{'numRemovedBytes': '1963749', 'p50FileSize': '1963749', 'p25FileSize': '1963749', 'numAddedFiles': '1', 'minFileSize': '1963749', 'p75FileSize': '1963749', 'numDeletionVectorsRemoved': '0', 'numAddedBytes': '1963749', 'maxFileSize': '1963749', 'numRemovedFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
3,2024-02-04 17:05:38.590000,null,null,UPDATE,"{'predicate': '[""isnull(director#40)""]'}",null,null,null,2,Serializable,False,"{'numRemovedBytes': '1960782', 'numAddedFiles': '1', 'scanTimeMs': '2915', 'numCopiedRows': '6172', 'executionTimeMs': '5820', 'numAddedChangeFiles': '0', 'numAddedBytes': '1963749', 'numUpdatedRows': '2634', 'numRemovedFiles': '1', 'rewriteTimeMs': '2899'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
2,2024-02-04 17:04:16.904000,null,null,SET TBLPROPERTIES,"{'properties': '{""delta.logRetentionDuration"":""interval 60 days"",""delta.deletedFileRetentionDuration"":""interval 14 days""}'}",null,null,null,1,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
1,2024-02-04 17:03:13.178000,null,null,CREATE OR REPLACE TABLE AS SELECT,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,0,Serializable,False,"{'numOutputRows': '8806', 'numOutputBytes': '1960782', 'numFiles': '1'}",null,Apache-Spark/3.4.1 Delta-Lake/2.4.0
0,2024-02-04 17:03:06.818000,null,null,CREATE OR REPLACE TABLE,"{'description': None, 'partitionBy': '[]', 'properties': '{}', 'isManaged': 'false'}",null,null,null,null,Serializable,True,{},null,Apache-Spark/3.4.1 Delta-Lake/2.4.0


### Partitioning

In [12]:
# Read the data as a DataFrame
df = (spark.read.format("json")
      .option("multiLine", "true")
      .load("../data/nobel_prizes.json"))

In [13]:
df.printSchema()

root
 |-- category: string (nullable = true)
 |-- laureates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- motivation: string (nullable = true)
 |    |    |-- share: string (nullable = true)
 |    |    |-- surname: string (nullable = true)
 |-- overallMotivation: string (nullable = true)
 |-- year: string (nullable = true)



In [14]:
# Write the data to a Delta Lake table with partitioning
(df.write.format("delta")
 .mode("overwrite")
 .partitionBy("year")
 .save("/opt/workspace/data/delta_lake/nobel_prizes"))

In [15]:
# Query the partitioned table
df = spark.read.format("delta").load("/opt/workspace/data/delta_lake/nobel_prizes")
df.show()

+----------+--------------------+-----------------+----+
|  category|           laureates|overallMotivation|year|
+----------+--------------------+-----------------+----+
| chemistry|[{Paul J., 246, "...|             null|1974|
| economics|[{Gunnar, 684, "f...|             null|1974|
|literature|[{Eyvind, 649, "f...|             null|1974|
|     peace|[{Seán, 532, "for...|             null|1974|
|   physics|[{Martin, 100, "f...|             null|1974|
|  medicine|[{Albert, 403, "f...|             null|1974|
| chemistry|[{Svante, 162, "i...|             null|1903|
|literature|[{Bjørnstjerne, 5...|             null|1903|
|     peace|[{Randal, 466, "f...|             null|1903|
|   physics|[{Henri, 4, "in r...|             null|1903|
|  medicine|[{Niels Ryberg, 2...|             null|1903|
| chemistry|[{Alfred, 174, "i...|             null|1913|
|literature|[{Rabindranath, 5...|             null|1913|
|     peace|[{Henri, 481, "fo...|             null|1913|
|   physics|[{Heike, 18, "for..

In [16]:
spark.stop()